<a href="https://colab.research.google.com/github/KepryDeSousa/Inteligencia-Computacional-I/blob/main/Relatorio_VI__MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relatorio VI: *Redes Multicamadas*
> Aluno : Jarom Kepri de Sousa M.
>
> Curso: Engenharia de Energias

Desafio:
> Implemente um sistema de controle para o motor CC com função de transferência da equação 18 do material anexo.
>
>através da estimação da função de transferência inversa da planta utilizando uma rede MLP.
>
>Defina os hiperparâmatros da rede, por tentativa e erro, deixando expresso quais o limites inferiores e superiores da faixa de variação escolhida pra essa busca, assim como o número de medições anteriores necessárias para se obter um bom resultado de modelagem.

 



In [110]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Dados dos parâmetros do motor CC
vi_max = 30.0  # Volts
Ia_max = 2.0  # Amperes
Ra = 3.0  # Ohms
La = 6.0e-3  # Henries
Ki = 50.0e-3  # Nm/A
T_nominal = 0.1  # Newton-metro
Jm = 40.0e-6  # Kg*m^2
Bm = 40.0e-6  # Kg*m^2/s
w_nominal = 300.0  # Radianos/s

# Faixa de variação para os dados de entrada e saída
vi_range = (0.0, vi_max)  # Volts
Ia_range = (0.0, Ia_max)  # Amperes
T_range = (0.0, T_nominal)  # Newton-metro
w_range = (0.0, w_nominal)  # Radianos/s

# Número de medições anteriores para obter um bom resultado de modelagem
num_medicoes_anteriores = 10

# Função para gerar dados de treinamento
def gerar_dados_treinamento(num_samples):
    vi = np.random.uniform(vi_range[0], vi_range[1], size=num_samples)
    Ia = np.random.uniform(Ia_range[0], Ia_range[1], size=num_samples)
    T = Ki * Ia
    w = (T - Bm * w_nominal) / Jm
    X = np.column_stack((vi, Ia))
    y = np.column_stack((T, w))
    return X, y

# Hiperparâmetros da rede MLP
num_epocas = 100
tamanho_lote = 32
taxa_aprendizado = 0.001
num_neuronios_camada_oculta = 50

# Criar a rede MLP
modelo = Sequential()
modelo.add(Dense(num_neuronios_camada_oculta, input_dim=2, activation='relu'))
modelo.add(Dense(2, activation='linear'))

# Compilar o modelo
modelo.compile(loss='mean_squared_error', optimizer='adam')

# Gerar dados de treinamento
num_samples_treinamento = 1000
X_treinamento, y_treinamento = gerar_dados_treinamento(num_samples_treinamento)

# Treinar a rede MLP
modelo.fit(X_treinamento, y_treinamento, epochs=num_epocas, batch_size=tamanho_lote, verbose=1)

# Avaliar o desempenho da rede MLP
num_samples_teste = 15
X_teste, y_teste = gerar_dados_treinamento(num_samples_teste)
score = modelo.evaluate(X_teste, y_teste, verbose=1)
print("Loss: ", score)


Epoch 1/100
32/32 [==============================] - 2s 1ms/step - loss: 678926.2500
Epoch 2/100
32/32 [==============================] - 0s 1ms/step - loss: 674209.1250
Epoch 3/100
32/32 [==============================] - 0s 1ms/step - loss: 669409.5625
Epoch 4/100
32/32 [==============================] - 0s 1ms/step - loss: 664206.7500
Epoch 5/100
32/32 [==============================] - 0s 1ms/step - loss: 658148.3750
Epoch 6/100
32/32 [==============================] - 0s 1ms/step - loss: 650770.6250
Epoch 7/100
32/32 [==============================] - 0s 2ms/step - loss: 642117.4375
Epoch 8/100
32/32 [==============================] - 0s 1ms/step - loss: 632305.3750
Epoch 9/100
32/32 [==============================] - 0s 2ms/step - loss: 621429.7500
Epoch 10/100
32/32 [==============================] - 0s 2ms/step - loss: 609482.8125
Epoch 11/100
32/32 [==============================] - 0s 2ms/step - loss: 596153.5625
Epoch 12/100
32/32 [==============================] - 0s 1ms/st

## Referencias:  
> https://www.citisystems.com.br/motor-cc/
>
>https://iaexpert.academy/2018/05/17/deep-learning-redes-neurais-com-keras-e-python/
>
